In [10]:
import numpy as np
import pandas as pd
import mediapipe as mp
import os
import cv2

In [11]:
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_hands = mp.solutions.hands

In [12]:
alphabets = list("ABCDEFGHIKLMNOPQRSTUVWXY")
print(alphabets)

['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y']


In [13]:
filename_name = "data.csv"

# Get the directory path of the file
parent_dir = os.path.dirname(file_path)

#chemin vers les données
data_dir = parent_dir + "/data"

# Print the directory path
print(data_dir)


NameError: name 'file_path' is not defined

In [ ]:
temp_dataset = []

for label in alphabets:
    dataset_directory_prefix = '.../Raw_data/data/'
    for dirname, _, filenames in os.walk(dataset_directory_prefix + label):
        for filename in filenames:
            pathname = os.path.join(dirname, filename)

            with mp_hands.Hands(
                static_image_mode=True,
                max_num_hands=1,
                min_detection_confidence=0.3
            ) as hands:

                image = cv2.flip(cv2.imread(pathname), 1)
                result = hands.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

                landmark_vertices_xyz = []
                if not result.multi_hand_landmarks:
                    continue

                for l in result.multi_hand_landmarks[0].landmark:
                    landmark_vertices_xyz.append(l.x)
                    landmark_vertices_xyz.append(l.y)
                    landmark_vertices_xyz.append(l.z)

                temp_dataset.append((*landmark_vertices_xyz, label))
                
    print(f'imported: {label}')
        
landmark_vertices_xyz_label = []
for idx in range(21):
    for char in list('xyz'):
        vertex_label = char+str(idx)
        landmark_vertices_xyz_label.append(vertex_label)
            
print(*landmark_vertices_xyz_label)
        
dataset = pd.DataFrame(temp_dataset, columns=[*landmark_vertices_xyz_label, 'target'])
dataset.head()

In [15]:
dataset.to_csv('data.csv', index=False)

In [16]:
from sklearn.model_selection import train_test_split

X = dataset.drop(['target'],axis=1).values
y = dataset['target'].values

# Choose your test size to split between training and testing sets:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

ValueError: With n_samples=0, test_size=0.25 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.